In [22]:
import re
from collections import defaultdict

from bs4 import BeautifulSoup, NavigableString

from clip import ClipboardHandler, Clip
from notion_clip import NotionClip
from notion_client import Client


In [23]:
with ClipboardHandler() as ch:
    print(ch.get_formatnames())
    # print(ch.set_html())

{49412: 'HTML Format', 13: 'standard_format', 49744: 'Chromium internal source URL', 16: 'standard_format', 1: 'standard_format', 7: 'standard_format'}


In [24]:
NOTION_ACCESS_TOKEN = "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"
notion = Client(auth=NOTION_ACCESS_TOKEN)

response = notion.users.list()

top_id = "177e700eb51580bfaf7bfaddd97cbc78"

# response = notion.blocks.children.list(block_id=top_id)

# for block in response.get("results", []):
#     print(block)

In [25]:
def extract_fragment(text):
    # 正規表現で最初の <!--StartFragment--> と最後の <!--EndFragment--> を見つける
    start_pattern = r'<!--StartFragment-->'
    end_pattern = r'<!--EndFragment-->'

    # 文字列全体にわたって両方のタグの位置を探す
    start_match = re.search(start_pattern, text)
    end_match = re.search(end_pattern, text)

    if start_match and end_match:
        # 開始タグの終了位置と終了タグの開始位置を取得
        start_pos = start_match.end()
        end_pos = text.rfind(end_pattern)
        
        # 外側のペアに囲まれた文字列を取得
        return text[start_pos:end_pos].strip()
    else:
        return text

In [26]:
clip = NotionClip()
# print(clip.get_formats())
# clip.disp()
html = clip["HTML_bin"].decode()
html = extract_fragment(html)


soup = BeautifulSoup(html, "lxml", preserve_whitespace_tags={"textarea", "p", "h1", "h2", "h3", "code", "strong"})
print(soup.prettify())



<html>
 <body>
  <h3><strong>1. <code>text_elem</code>の構造と初期化の統一</strong></h3>
  <p>1つ目のコードでは、<code>text_elem</code>の初期化時に<code>defaultdict</code>を使用しており、<code>text_elem["type"]</code>を別で定義しています。一方、2つ目のコードでは<code>text_elem</code>を専用の関数内で生成しています。</p>
  <h4>
   修正案:
  </h4>
  <p>1つ目のコードの<code>text_elem</code>の初期化を2つ目の構造に</p>
 </body>
</html>



In [27]:
# type(clip["Notion"])

In [28]:
page_id = "177e700e-b515-8045-bf48-cc725679365e"

In [29]:
def append_object(parent_id, object):
    try:
        response = notion.blocks.children.append(parent_id, children=[object])
        if "results" in response and response["results"]:
            block_id = response["results"][0]["id"]
            print(f"Block created with ID: {block_id}")
            return block_id
        else:
            print(f"Failed to append block: {response}")
            return None
    except Exception as e:
        print(f"Error appending block: {e}")
        return None
        
def make_block(block, block_type):
    rich_text = process_inline_elements(block)
    block_content = {
        # "object": "block",
        "type": block_type,
        block_type: {"rich_text": rich_text},
    }
    
    return block_content

def process_inline_elements(element, text_elem=None):
    rich_text = []
    
    if text_elem is None:
        text_elem = defaultdict(dict)
        text_elem["type"] = "text"

    # if text_elem is None:
    #     text_elem = {
    #         "type": "text",
    #         "annotations": {
    #             "bold": False,
    #             "italic": False,
    #             "strikethrough": False,
    #             "underline": False,
    #             "code": False,
    #             "color": "default",
    #         },
    #         "plain_text": None,
    #         "href": None,
    #     }

    # タグごとに処理
    if element.name in ["strong", "b"]:
        text_elem["annotations"]["bold"] = True
    if element.name in ["em", "i"]:
        text_elem["annotations"]["italic"] = True
    if element.name in ["u", "ins"]:
        text_elem["annotations"]["underline"] = True
    if element.name in ["s", "strike"]:
        text_elem["annotations"]["strikethrough"] = True
    if element.name == "code":
        text_elem["annotations"]["code"] = True
        text_elem["annotations"]["color"] = "red"


    if isinstance(element, (str, NavigableString)):
        # 文字列またはNavigableStringの場合
        content = str(element).replace('\n', ' ').strip()
        if content:
            text_elem[text_elem["type"]]["content"] = content
            rich_text.append(text_elem.copy())

    elif element.name == "math" and not element.has_attr("display"):
        expression = extract_math_block(element)
        text_elem["type"] = "equation"
        text_elem[text_elem["type"]]["expression"] = expression
        rich_text.append(text_elem.copy())

    else:
        # 子要素があるか確認
        has_children = any(not isinstance(child, (str, NavigableString)) for child in element.children)

        if not has_children and element.string:
            # 子要素がなく、テキストノードの場合
            content = element.string.replace('\n', ' ').strip()
            if content:
                text_elem[text_elem["type"]]["content"] = content
                rich_text.append(text_elem.copy())
        else:
            # 子要素の再帰処理
            for child in element.children:
                if child.name == "p":
                    return rich_text
                elif child.name not in ["h1", "h2", "h3", "ul", "ol", "li", "div", "pre"]  or  child.name == "math" and child.has_attr("display"):
                    rich_text.extend(process_inline_elements(child, text_elem))

    return rich_text

def extract_code_block(block):
    language_div = block.find("div", class_="flex items-center text-token-text-secondary px-4 py-2 text-xs font-sans justify-between rounded-t-md h-9 bg-token-sidebar-surface-primary dark:bg-token-main-surface-secondary select-none")
    language = language_div.get_text(strip=True)
    if not language in [ "abap", "agda", "arduino", "ascii art", "assembly", "bash", "basic", "bnf", "c", "c#", "c++", "clojure", "coffeescript", "coq", "css", "dart", "dhall", "diff", "docker", "ebnf", "elixir", "elm", "erlang", "f#", "flow", "fortran", "gherkin", "glsl", "go", "graphql", "groovy", "haskell", "hcl", "html", "idris", "java", "javascript", "json", "julia", "kotlin", "latex", "less", "lisp", "livescript", "llvm ir", "lua", "makefile", "markdown", "markup", "matlab", "mathematica", "mermaid", "nix", "notion formula", "objective-c", "ocaml", "pascal", "perl", "php", "plain text", "powershell", "prolog", "protobuf", "purescript", "python", "r", "racket", "reason", "ruby", "rust", "sass", "scala", "scheme", "scss", "shell", "smalltalk", "solidity", "sql", "swift", "toml", "typescript", "vb.net", "verilog", "vhdl", "visual basic", "webassembly", "xml", "yaml", "java/c/c++/c#", "notionscript" ]:
        language = "plain text"

    code_element = block.find("code", class_=re.compile("^!whitespace-pre hljs language-"))
    code = code_element.text.strip()

    return language, code

def extract_math_block(block):
    math_block = block.find('annotation', {'encoding': 'application/x-tex'})
    expression = math_block.text

    return expression

def html_to_notion(parent_id, block):

    if isinstance(block, str):
        return
    block_id = None
    
    if block.name == "p":
        block_type = "paragraph"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
        block_type = f"heading_{min(int(block.name[1]), 3)}"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name == "li":
        block_type = None
        parent_tag = block.parent.name if block.parent else None
        if block.find(True):  # 直下にタグがある場合
            first_child = block.contents[0]  # 直下の最初のタグを取得
            if first_child.name == "p":
                first_child.unwrap() # <p>タグのタグのみを削除
        if parent_tag == "ol":
            block_type = "numbered_list_item"
        else:
            block_type = "bulleted_list_item"
        block_content = make_block(block, block_type)
        block_id = append_object(parent_id, block_content)

    elif block.name == "pre":
        # try:
            code_language, code = extract_code_block(block)
            block_content = {
              "type": "code",
              "code": {
                "caption": [],
                    "rich_text": [{
                    "type": "text",
                    "text": {
                    "content": code
                    }
                }],
                "language": code_language
              }
            }
            block_id = append_object(parent_id, block_content)
        # except AttributeError:
            # return

    elif block.name == "math" and block.has_attr('display'):
        expression = extract_math_block(block)
        block_content = {
          "type": "equation",
          "equation": {
            "expression": expression,
            }
        }
        block_id = append_object(parent_id, block_content)

    elif block.name in ["html", "ul", "ol"]:
        block_id = parent_id
        
    elif block.name in ["body"]:
        block_id = parent_id
        # body直下の内容がブロック要素を含まないとき、出力できるように
        # 要改善 もう一周回す？
        if not any(((child.name in ["h1", "h2", "h3", "ul", "ol", "li", "div", "pre", "math"]), child.has_attr("display")) for child in block.children if child.name):
            block.name = "p"
            html_to_notion(block_id, block)
            return
            
    else:
        block_id = parent_id
            
        
    for child in block.children:
        html_to_notion(block_id, child)


In [30]:
html_ele = BeautifulSoup(html, "lxml").body

print(html_ele)
html_to_notion(page_id, html_ele)

# print(list(html.children))

# print(html)

<body><h3><strong>1. <code>text_elem</code>の構造と初期化の統一</strong></h3><p>1つ目のコードでは、<code>text_elem</code>の初期化時に<code>defaultdict</code>を使用しており、<code>text_elem["type"]</code>を別で定義しています。一方、2つ目のコードでは<code>text_elem</code>を専用の関数内で生成しています。</p><h4>修正案:</h4><p>1つ目のコードの<code>text_elem</code>の初期化を2つ目の構造に</p></body>
Block created with ID: 187e700e-b515-812c-8828-ea4764e93f93
Block created with ID: 187e700e-b515-8155-a931-f9f0f45003fd
Block created with ID: 187e700e-b515-81b8-9bb7-c7b61ebeb529
Block created with ID: 187e700e-b515-81c4-a75f-d5e441189405
